<a href="" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



<div style="text-align:center"><img src="Figures/thumb.PNG" /></div>

# Training StyleGAN2 on the cropped dataset in Google CoLab  

### Google Drive setups

One of the main requirements for starting is google drive. We're going to use G-DRIVE to store our training data and trained neural networks. we built some folders with the path of follows:
```
content/drive/MyDrive/gan 
``` 

### Managing GPU

There are two options for training GANs with Google Colab, Google Colab Free or Pro. We will go with the pro one because it has advantages like better GPU, longer runtime, and timeouts, and most important, it will not disconnect before 24 hours. In using the free version of google colab, you should make sure that you run the notebook with GPU runtime. For doing this, go to "Runtime" in the bar and select the "change the runtime type" option. Then, put the Hardware accelerator on GPU and save it.

In [ ]:
!nvidia-smi

### Steps of the code

1) connecting Colab to GDRIVE, to save the snapshots there

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

2) creating needful directories in GDRIVE

In [ ]:
!mkdir drive/MyDrive/
!mkdir drive/MyDrive/gan
!mkdir drive/MyDrive/gan/images
!mkdir drive/MyDrive/gan/dataset
!mkdir drive/MyDrive/gan/experiments

3) [Download the __cropped__ dataset](https://github.com/mahdi-darvish/GAN-augmented-pet-classifier#dataset)

In [ ]:
# !wget [insert dataset's download link here]

4) Extract the images

In [ ]:
!tar xvf  images.tar.gz -C drive/MyDrive/gan/

In [ ]:
!mkdir drive/MyDrive/gan/image

5) Selecting a subset of the dataset.

Every breed involves 200 images, so you can consider that and fill the percentage variable according to your need and train the the model on that amount of data.

In [ ]:
import shutil
import os

source = "drive/MyDrive/gan/images"
destination = "drive/MyDrive/image"
percentage = None    #set this parameter according to your need
files_list = os.listdir(source)
lst = []
for files in sorted(files_list):
    x = files.rsplit('_', 1)[0]
    lst.append(x)
for l in list(set(lst)):
    cnt = 0
    for i in range(1, 100):
    try:
        shutil.copy(source + '/' + '{}_{}.jpg'.format(l, i), destination + '/' + '{}_{}.jpg'.format(l, i))
        cnt += 1
    except:
        print('{}_{}.jpg'.format(l, i))
    if cnt == 20:
        print('selected [] images from [] category'.format(percentage, l))
        break

6) unzipping the training images in "images" folder

In [ ]:
!unzip drive/MyDrive/images.zip -d drive/MyDrive/gan/images

7) Number of total training images and establishig the exact path of images

In [ ]:
!ls drive/MyDrive/gan/images | wc -l

8) Installing NVIDIA StyleGAN2 ADA PyTorch

In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

9) This code takes a python file and converts the first given path to tensors into the second given direction. We can face an error in this part in conditions like inconsistent types and variant sizes of images. We convert all the images into jpg type and resize them to 128 x 128 pixels.

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/images/circuit --dest /content/drive/MyDrive/data/gan/dataset/circuit

10) setting the version of the torch, torchvision,  torchaudio to become compatible with Style-GAN2ada

In [ ]:
!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

11) to clear out the newly created dataset in case something went wrong 

In [ ]:
!rm -rf /root/.cache/torch_extensions/*

12) making all the images to the exact dimensions and color depth

In [ ]:

import os
from tqdm.notebook import tqdm
from PIL import Image
from os import listdir

IMAGE_PATH = 'drive/MyDrive/gan/images'
files = [f for f in listdir(IMAGE_PATH) if os.path.isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
    file2 = os.path.join(IMAGE_PATH,file)
    img = Image.open(file2)
    sz = img.size
    if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
    elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
    else:
    base_size = sz


13) Converting all the images to jpg type

In [ ]:
from PIL import Image
import os, sys

path = "drive/MyDrive/gan/images"

for item in os.listdir(path):
    im = Image.open(path + '/' +  item)
    if im.mode != "RGB" :
        im = im.convert("RGB")
    imResize = im.resize((128,128))
    imResize.save(path + '/' +  item )



### Perform Initial Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/gan/experiments"
DATA = "/content/drive/MyDrive/gan/dataset"
SNAP = 20

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

### Resume Training

14) removing the last trained network into the experiment folder for continuing the training

In [ ]:
!rm drive/MyDrive/dogs/data/gan/experiments/network-snapshot* drive/MyDrive/dogs/data/gan/experiments

In [ ]:
import os

EXPERIMENTS = "/content/drive/MyDrive/gan/experiments/"
NETWORK = "network-snapshot-000480.pkl"
RESUME = os.path.join(EXPERIMENTS, NETWORK)
DATA = "/content/drive/MyDrive/gan/dataset"
SNAP = 20

cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}

Copyright 2021 by [MIT license](https://github.com/mahdi-darvish/GAN-augmented-pet-classifier/blob/main/LICENSE)